In [ ]:
## Import packages
# EarthEngine
import ee
from ee import batch # for data importation
# Data Manipulation
import pandas as pd
# Plot Manipulation
import matplotlib.pyplot as plt

## Connection to the Earth Engine (need personal account)
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=XNMf7Uyg5hXoiRzPTxpzdVQ-8aOZ0TdLm4uJ0ixpaIA&tc=AnNME3x-Q-ynXXRVnrsVoCV9RzWgGwsdKYUQcCMcVsQ&cc=kQx3Svknj4vunflJFP3FfvleYODhLkUY4zHBhqAn3gE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl7-63Ufm817RVzYzabJkv-KTjXhb7bZ0SD4-vYnH0mI_FsIU0xkng

Successfully saved authorization token.


In [ ]:
# Define the time period
start_date = '1980-01-01'
end_date = '2023-01-01'

# Create an ImageCollection for the ERA5-Land dataset.
dataset = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
            .filter(ee.Filter.date(start_date, end_date))
dataset_region = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
region = dataset_region.filter(ee.Filter.Or(ee.Filter.eq("ADM0_NAME", "France")))
reg = region.geometry()

# Select the 'evaporation_from_open_water_surfaces_excluding_oceans' band.
# "temperature_2m", "soil_temperature_level_1", "soil_temperature_level_4", "lake_bottom_temperature", "lake_ice_depth", "volumetric_soil_water_layer_1", "snow_depth", "surface_net_solar_radiation_sum", "evaporation_from_vegetation_transpiration_sum", "u_component_of_wind_10m", "v_component_of_wind_10m", "surface_pressure", "total_precipitation_sum", ""
total_precipitation = dataset.select('evaporation_from_bare_soil_sum')

# Define your location. (Paris [2.3522, 48.8566])
#location = ee.Geometry.Point([6.3166654, -10.7999968])

# Function to calculate monthly sum.
#def calc_daily_sum(img):
#    date = img.date().format('YYYY-MM-dd')
#    daily_img = img.reduceRegion(ee.Reducer.sum(), location, 1000).set('date', date)
#    return ee.Feature(None, daily_img)

def calc_daily_sum(img):
    date = img.date().format('YYYY-MM-dd')
    daily_img = img.reduceRegion(ee.Reducer.sum(), reg, 1000).set('date', date)
    return ee.Feature(None, daily_img)

# Apply the function to each image in the ImageCollection.
daily_sum = total_precipitation.map(calc_daily_sum)

# Convert the ImageCollection to a FeatureCollection.
daily_sum_fc = ee.FeatureCollection(daily_sum)

# Export the data to a CSV file.
task_config = {
    'collection': daily_sum_fc,
    'description': 'daily_Sum_EvapSoil',
    'folder': 'SeriesTempData',  # Replace with your Google Drive folder name
    'fileFormat': 'csv'
}

task = ee.batch.Export.table.toDrive(**task_config)
task.start()

In [ ]:
task.status()

{'state': 'RUNNING',
 'description': 'daily_Sum_EvapSoil',
 'creation_timestamp_ms': 1701611346111,
 'update_timestamp_ms': 1701611350990,
 'start_timestamp_ms': 1701611350957,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 1,
 'id': '7FWBKDJGJIQ5HFSXQCTH7OUU',
 'name': 'projects/earthengine-legacy/operations/7FWBKDJGJIQ5HFSXQCTH7OUU'}

In [ ]:
print(ee.batch.Task.list())

[<Task 4AYIMHRBZ6JPDP2QC5DZJSPS EXPORT_FEATURES: daily_Sum_EvapSoil (FAILED)>, <Task DLQXHGXFJRV5RWTEG3GYSY2J EXPORT_FEATURES: daily_Sum_Temp2M (COMPLETED)>, <Task YAHCPH5FWKXFPBUQ4GUGHLTU EXPORT_FEATURES: daily_Sum_TotPrecipitation (COMPLETED)>, <Task 2RGBDNJDV2PRBZPXIFQ3KUSZ EXPORT_FEATURES: daily_Sum_TotPrecipitation (COMPLETED)>, <Task K6T4TEOQMELFAVSH7PZ2KPYJ EXPORT_FEATURES: daily_Sum_SurfacePressure (COMPLETED)>, <Task 7PBJU6MSWM7QUNOSPOYNMPWZ EXPORT_FEATURES: daily_Sum_vWind (COMPLETED)>, <Task S2RHHHARJGKTFGL7WFMC4XPV EXPORT_FEATURES: daily_Sum_uWind (COMPLETED)>, <Task RS3V2PJFGNUMZZC2L3VCTREC EXPORT_FEATURES: daily_Sum_Radiation (COMPLETED)>, <Task NJGIJJDDKMPZSZBLDPMN5FDV EXPORT_FEATURES: daily_Sum_LakeBottomTemp (COMPLETED)>, <Task WEICWKS272F5CT7RPPNXRJVK EXPORT_FEATURES: daily_Sum_SoilTemp4 (COMPLETED)>, <Task JDGEKGU2HUK33ODBYF32YAWT EXPORT_FEATURES: daily_Sum_SoilTemp1 (COMPLETED)>, <Task FEWXKFF7XME6NOHS5PVZAPLC EXPORT_FEATURES: daily_Sum_SkinTemp (COMPLETED)>, <Task 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive
